In [11]:
from src.indexing_utils import load_docs, create_docstore
from ved_src import retreival_utils
import os

In [2]:
nodes = load_docs('../../../storage/docstore/swapped_summary_nodes', return_docstore=False)

In [6]:
for node in nodes:
    node.excluded_embed_metadata_keys.extend(['summary', 'is_summary_swapped'])

In [12]:
create_docstore(nodes=nodes, save_dir='../../../storage/docstore', store_name='swapped_summary_nodes')

In [42]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings

Settings.embed_model = GeminiEmbedding(model_name='models/embedding-001')
Settings.llm = Gemini(model_name='models/gemini-pro', temperature=0.5)

In [16]:
# load the documents and create the index
index = VectorStoreIndex(nodes=nodes, embed_model=Settings.embed_model, show_progress=True)

# store it for later
index.storage_context.persist(persist_dir='../../../storage/vectorstore/summary_embeddings_vectorstore')

Generating embeddings:   0%|          | 0/427 [00:00<?, ?it/s]

In [29]:
store = load_docs('../../../storage/docstore/swapped_summary_nodes', return_docstore=True)

In [43]:
query_engine = index.as_query_engine(llm=Settings.llm)

In [56]:
response = query_engine.query("How does ayurveda deal with illnesses like HIV AIDS")

In [57]:
from llama_index.legacy.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)

Final Response: Ayurveda attributes HIV/AIDS to immunodeficiency and
classifies it as "Dhaatugata Jvara," a condition where the immune
system is weakened. For asymptomatic HIV/AIDS patients, Ayurveda
recommends a tissue-building approach (Santarpan`a Chikitsaa) to
restore the immune system and prevent opportunistic infections. Herbs
like Ashwagandha, Amla, and Guduchi are used in this therapy.
S`hodhana therapy (purification) is advised only for asymptomatic
patients and should be combined with Santarpan`a Chikitsaa. The choice
of herbs depends on the affected body system (Srotas).
______________________________________________________________________
Source Node 1/2
Node ID: f0153eec-5bad-435c-be45-fcd89561dc29
Similarity: 0.7794220610267687
Text: Ayurvedic medicine, a traditional Indian system of medicine,
emphasizes the balance of three doshas: Vata, Pitta, and Kapha. These
doshas are believed to govern various functions in the body, and
imbalances can lead to disease. Ayurveda also

In [49]:
store.get_document("0723fed9-b35c-4696-8cb1-387e39aa5c4d").metadata['summary']

'Other Description:'